<a href="https://colab.research.google.com/github/JoesHage/DrunkEye/blob/main/Food_Recognition_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food Item Recognition**

We will be using the following 3 models to test which one of them is ideal to our current scenario.
*   **SVM**
*   **Random Forest**
*   **CNN**

We need to classify images into 1 of 9 classes using these models.

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os

# Get the path to the data folder
data_folder = "/content/drive/MyDrive/Food Item Recognition From Images1 (1)/data"

# Get a list of all folders in the data folder
folders = pd.Series(os.listdir(data_folder)).sort_values()

# Print the list of folders
print(folders)


2               1
4               2
1               3
5               4
7               5
6               6
8               7
9               8
0               9
3    category.txt
dtype: object


# **I. Support Vector Machines**


# **II. Random Forest**

In [ ]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score

def extract_features(image_path, image_size=(100, 100)):
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Unable to load image '{image_path}'")
            return None
        image = cv2.resize(image, image_size)
        # Convert image to grayscale and flatten the array
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return gray_image.flatten()
    except Exception as e:
        print(f"Error: An exception occurred while processing image '{image_path}': {e}")
        return None

X = []
y = []

with open(os.path.join(data_folder, 'category.txt'), 'r') as category_file:
    next(category_file)  # Skip the header line
    for line in category_file:
        category_id, category_name = line.strip().split('\t')
        category_id = int(category_id)
        category_folder = os.path.join(data_folder, str(category_id))
        # Iterate over images in the category folder
        for image_file in os.listdir(category_folder):
            image_path = os.path.join(category_folder, image_file)
            # Extract features and append to X
            features = extract_features(image_path)
            X.append(features)
            # Append corresponding label to y
            y.append(category_id)

X = np.array(X)
y = np.array(y)

#Split data into training and testing data, 0.8 and 0.2 respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

#Train Random forest classifier on training data
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=32)
rf_classifier.fit(X_train, y_train)

#Evaluation
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

#Using 10 fold cross validation
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=32)
cv_scores = cross_val_score(rf_classifier, X, y, cv=5)
print(f"CV Accuracy Scores: {cv_scores}")
print(f"Average CV Accuracy: {np.mean(cv_scores)}")

#Using hyperParameter tuning to find the ideal estimator.
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=32)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=10, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(grid_search.best_score_))

#Testing the best model we got from the grid search's estimator
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy: {:.2f}".format(accuracy))

Accuracy: 0.5910418695228822
CV Accuracy Scores: [0.50827653 0.60564752 0.54527751 0.59746589 0.50487329]
Average CV Accuracy: 0.552308147844457
Fitting 10 folds for each of 108 candidates, totalling 1080 fits


Using only the testing, training data we check the accuracy of the model and it gives a moderate 0.58.
Now we move on and add 10 fold cross validation for a more accurate accuracy score: 0.54.
We now want to improve the model more by using hyperparameter tuning and getting the best parameters for a model.

# **III. Convolutional neural networks**